# Rechercher les meilleurs hyper-paramètres

In [8]:
import hashlib
import os
import shutil
import time
from typing import Callable
from matplotlib import pyplot as plt
import numpy as np

import jax
import jax.numpy as jnp
import jax.random as jr
import optax
import pickle
from datetime import datetime

from dataclasses import dataclass

## Data

### Un problème de regression

On veut faire coller un modèle à une fonction toute simple.

In [9]:
n_data=20_000
X = jnp.linspace(0, 1, n_data)[:, None]
Y = jnp.sin(10 * X)
nb_data_train = int(n_data * 0.8)
idx=np.random.permutation(n_data)
X_=X[idx]
Y_=Y[idx]
X_train = X_[:nb_data_train]
Y_train = Y_[:nb_data_train]
X_val = X_[nb_data_train:]
Y_val = Y_[nb_data_train:]

In [10]:
fig,ax=plt.subplots()
ax.plot(X,Y);

### Distributeur de batch

On va distribuer les données avec une double boucle. Une boucle sur les époques. A chaque époque les données sont réparties dans les batchs.

In [11]:
def batchs_for_one_epoch(X_all,Y_all,batch_size):
    nb_batches=len(X_all)//batch_size
    shuffle_index=np.random.permutation(len(X_all))
    X_all_shuffle=X_all[shuffle_index]
    Y_all_shuffle=Y_all[shuffle_index]
    for i in range(nb_batches):
        yield X_all_shuffle[i*batch_size:(i+1)*batch_size],Y_all_shuffle[i*batch_size:(i+1)*batch_size]

In [12]:
X_all=jnp.zeros([1003,2])
Y_all=jnp.ones([1003,1])

iterator_for_one_epoch=batchs_for_one_epoch(X_all,Y_all,100)

for x,y in iterator_for_one_epoch:
    print(x.shape,y.shape)

Remarque: les 3 données de la fin ne sont pas distribuée. C'est pas grave, car le shuffle change l'ordre à chaque époque.

## Modèle et fonctions d'entrainement

Un réseau neuronal est une fonction paramétrique $f(\theta,x)$. En jax, on l'implémente très explicitement: en se donnant
* la fonction `model_init(rkey)` qui renvoie un jeu initial de paramètre noté `params`, ou `model_params` ou $\theta$  
* et la fonction `model_call(params,x)` =$f(\theta,x)$.

In [13]:
def model_fnm(layer_widths,activation_fn):

    def model_init(rkey):
        params = []
        for n_in, n_out in zip(layer_widths[:-1], layer_widths[1:]):
            rk,rkey=jr.split(rkey)
            params.append(
                {"weight":jr.normal(rk,shape=(n_in, n_out))*jnp.sqrt(2/n_in),
                "bias":jnp.zeros([n_out])})
        return params

    def model_apply(params, inp):
        *hidden, last = params
        for layer in hidden:
            inp = activation_fn(inp @ layer['weight'] + layer['bias'])
        return inp @ last['weight'] + last['bias']

    return model_init,model_apply

### De l'Hyper-paramètre au modèle

In [14]:
def make_model(hyper_param):

    dim_in=1
    dim_out=1

    dim_hidden=hyper_param['dim_hidden']
    n_layer=hyper_param['n_layer']
    activation_name=hyper_param['activation_name']


    layer_widths=[dim_in]+[dim_hidden]*(n_layer-1)+[dim_out]


    if activation_name=="relu":
        activation_fn=jax.nn.relu
    else:
        activation_fn=jax.nn.tanh
    #on pourrait mettre plein d'autre choix de fonctions d'activation

    return model_fnm(layer_widths,activation_fn)

Ensuite, il faut trouver le bon paramètre $\theta$ pour que le réseau neuronal colle aux données. Cette recherche se fait à l'aide d'une descente de gradient.

Mais pour réussir au mieux, il faut aussi bien calibrer l'architecture du réseau de neurone (`n_layer`, `dim_hidden`, `activation_name`) paramètres de l'optimisation (`learning_rate`, `batch_size`, etc.). On nommera tous ces paramètres "hyper-paramètres" pour les différencier des paramètres du modèles.




Nous cherchons à organiser notre code pour pouvoir tester différents modèles et différents modes d'apprentissages. On aimerait trouver le bon équilibre entre "tout automatiser" et  "tout faire à la main".

On aimerait aussi que les paramètres des modèles entrainés soient sauvegardés pour pouvoir prolonger un entrainement. Et on veut aussi se souvenir de tous les hyper-paramètres qui ont été testé.

### fonction loss et update

In [15]:
def jit_creator(model_apply,optimizer):
    @jax.jit
    def loss_compute(params, X,Y):
        Y_pred=model_apply(params,X)
        return jnp.mean((Y_pred-Y)**2)

    @jax.jit
    def update_model_param(optimizer_state, model_param, X,Y):
        grads = jax.grad(loss_compute)(model_param, X,Y)
        updates, optimizer_state = optimizer.update(grads, optimizer_state)
        #here the model_param is modified
        model_param = optax.apply_updates(model_param, updates)
        #c'est idem que faire la somme des feuilles des pytrees model_param et updates comme ceci:
        #model_param = jax.tree.map(lambda x,y:x+y, model_param, updates)
        return optimizer_state, model_param

    return loss_compute,update_model_param

## L'Agent

J'utilise le mot 'Agent' pour un objet qui permet l'entrainement.

J'ai l'impression que maintenant la plupart des gens utilisent le mot 'Trainer'.


Appelons la technique que l'on va utiliser 'ful-folder'. Tout ce qui est produit durant l'entrainement est sauvé dans un folder.

### Fonctions de sauvegarde

In [16]:
def save_as_pickle(file_name,serializable):
    pickle.dump(serializable,open(file_name,"wb"))
def load_from_pickle(file_name):
    return pickle.load(open(file_name,"rb"))
def save_as_str(file_name,serializable):
    with open(file_name, "wt") as f:
        f.write(str(serializable))
def load_from_str(file_name):
    with open(file_name, "rt") as f:
        res = eval(f.read())
    return res

### L'Agent en personne

In [17]:
@dataclass
class AgentMiniResult:
    hyper_param:dict
    best_loss:float
    model_param:dict
    model_call:Callable


class AgentMini:

    @staticmethod
    def load(folder):
        assert os.path.exists(folder),f"folder:{folder} does not exist"
        model_param = load_from_pickle(f"{folder}/model_param")
        best_loss = load_from_str(f"{folder}/best_loss")
        model_param=load_from_pickle(f"{folder}/model_param")
        hyper_param=load_from_str(f"{folder}/hyper_param")
        _, model_call = make_model(hyper_param)
        return AgentMiniResult(hyper_param,best_loss,model_param,model_call)

    @staticmethod
    def train(folder,hyper_param,n_epoch,verbose):

        model_init, model_call = make_model(hyper_param)
        optimizer = optax.adam(hyper_param["learning_rate"])
        batch_size = hyper_param["batch_size"]

        if os.path.exists(folder):
            if verbose:
                print(f"Existing folder:{folder}, we load model_param and best_loss from if")
            model_param = load_from_pickle(f"{folder}/model_param")
            best_loss =load_from_str(f"{folder}/best_loss")

        else:#c'est la première fois qu'on teste cet hyper_paramètre.
            os.makedirs(folder, exist_ok=True)
            if verbose:
                print(f"New folder:{folder}, model_param are randomly initialized")
            model_param=model_init(jr.key(0))
            best_loss=1e10#l'infini ou presque
            save_as_pickle(f"{folder}/model_param", model_param)
            save_as_str(f"{folder}/best_loss", best_loss)

        save_as_str(f"{folder}/hyper_param", hyper_param)
        optimizer_state=optimizer.init(model_param)
        loss_compute, update_model_param=jit_creator(model_call,optimizer)

        for _ in range(n_epoch):
            for x,y in batchs_for_one_epoch(X_train,Y_train,batch_size):
                optimizer_state, model_param = update_model_param(optimizer_state, model_param, x,y)

            val_loss=float(loss_compute(model_param, X_val, Y_val))
            if val_loss <= best_loss:
                best_loss=val_loss
                if verbose:
                    print(f"⬊{val_loss:.3g}", end="")
                save_as_pickle(f"{folder}/model_param",model_param)
                save_as_str(f"{folder}/best_loss",best_loss)
            else:
                if verbose:
                    print(".",end="")
        if verbose:
            print("| end of the optimization loop.")
        return best_loss

## Les entrainements

### Entrainement pour 1 hyper-paramètre



In [18]:
folder="data/mon_premier_test"
#Pour que ce ne soit pas un ré-entrainement.
shutil.rmtree(folder,ignore_errors=True)
hyper_param = {"n_layer": 2, "dim_hidden": 32,"batch_size":512,"learning_rate":1e-3,"activation_name":"relu"}
AgentMini.train(folder,hyper_param,20,verbose=True)

In [19]:
def eval_training(folder):
    result=AgentMini.load(folder)
    fig, axs = plt.subplots(1, 1, sharex="all")
    Y_pred=result.model_call(result.model_param,X)
    axs.plot(X,Y_pred)
    axs.plot(X,Y)
    fig.tight_layout()
    plt.show()

In [20]:
eval_training(folder)

⇑ pas terrible

### Une rechercher en grille

In [29]:
def manual_loop(mother_folder):
    # noinspection PyDictCreation
    hyper_params = [
        {"n_layer": 2, "dim_hidden": 32},
        {"n_layer": 3, "dim_hidden": 32},
        {"n_layer": 4, "dim_hidden": 32},
        {"n_layer": 2, "dim_hidden": 64},
        {"n_layer": 3, "dim_hidden": 64},
        {"n_layer": 4, "dim_hidden": 64}
    ]
    for hyper_param in hyper_params:
        hyper_param["batch_size"] = 512
        # noinspection PyTypeChecker
        hyper_param["learning_rate"] = 1e-3
        # noinspection PyTypeChecker
        hyper_param["activation_name"] = "relu"

    for i,hyper_param in enumerate(hyper_params):
        folder=f"{mother_folder}/test_{i}"
        shutil.rmtree(folder,ignore_errors=True)
        AgentMini.train(folder,hyper_param,40,verbose=True)

In [30]:
manual_loop("data/manual_loop")

In [23]:
def hyper_param_to_str(hyper_param):
    return f"bs:{hyper_param['batch_size']},n_lay:{hyper_param['n_layer']},lay_size:{hyper_param['dim_hidden']},lr:{hyper_param['learning_rate']:.3g}"


In [31]:
def eval_trainings(mother_folder):
    folders=list(os.listdir(mother_folder))
    loss_results=[]
    for i,_folder in enumerate(folders):
        folder=f"{mother_folder}/{_folder}"
        result=AgentMini.load(folder)
        loss_results.append((result.best_loss,result))
    loss_results=sorted(loss_results,key=lambda a:a[0])
    results=[a[1] for a in loss_results]
    ni = 3
    some_results=[results[0],results[len(results)//2],results[-1]]
    fig, axs = plt.subplots(ni, 1, sharex="all", figsize=(6, ni * 4))
    titles=["best","intermediate","worst"]

    for i,result in enumerate(some_results):
        Y_pred=result.model_call(result.model_param,X)
        axs[i].plot(X,Y_pred)
        axs[i].plot(X,Y)
        axs[i].set_title(titles[i]+':'+hyper_param_to_str(result.hyper_param))
    fig.tight_layout()
    plt.show()

In [32]:
eval_trainings("data/manual_loop")

***A vous:*** Refaite ces entrainements avec le fonction d'activation sigmoid pour voir la différence.

### Une recherche aléatoire

In [35]:
def random_loop(mother_folder):
    # noinspection PyDictCreation


    i=0
    for activation_name in ["relu","tanh"]:
        for _ in range(10):
            i+=1
            hyper_param = {}
            hyper_param["dim_hidden"]=np.random.choice(a=[32,64,128])
            hyper_param["n_layer"]=np.random.choice(a=[2,3,4,5])
            hyper_param["batch_size"] = 512
            hyper_param["learning_rate"] = np.random.choice(a=[1e-2,1e-3,1e-4])
            hyper_param["activation_name"] = activation_name

            folder=f"{mother_folder}/random_test_{i}"
            shutil.rmtree(folder,ignore_errors=True)

            AgentMini.train(folder,hyper_param,40,verbose=True)

In [36]:
random_loop("data/random_loop")

In [37]:
eval_trainings("data/random_loop")

### Utilisation d'une libraire d'optimisation

Hyper-op est une librairie d'optimisation bayesienne. Ce genre d'optimisation est utile lorsque vous essayez de trouver le minimum (ou le maximum) d'une fonction dont l'évaluation prend du temps et dont vous ne connaissez pas la forme exacte (la "boîte noire").

Ici on cherche le minimum de la fonction qui à un hyper-paramètre associe la 'best-loss' obtenue pendant l'entrainement.

In [26]:
def hyperop_loop(mother_folder):
    n_epoch=20
    def objective(hyper_param):
        #un nome de fichier par rapport au temps
        #attention,il faut que les trainings durent plus d'une seconde
        n_second=str(time.time()).split(".")[0][5:]
        folder=f"{mother_folder}/test_{n_second}"
        return AgentMini.train(folder,hyper_param,n_epoch,False)

    from hyperopt import hp,fmin, tpe, space_eval
    space = {
        'batch_size': hp.choice('batch_size', [128,256,512]),
        'n_layer': hp.choice('n_layer', [2,3,4]),
        'dim_hidden': hp.choice('layer_size', [64,128,256]),
        'activation_name':hp.choice('activation_name',['relu','tanh']),
        'learning_rate':hp.loguniform('learning_rate',np.log(5e-4),np.log(1e-2))
    }
    best = fmin(objective, space, algo=tpe.suggest, max_evals=10)
    print("best hyper_parameter found:")
    print(space_eval(space, best))

In [27]:
hyperop_loop("data/hyperop_loop")

In [28]:
eval_trainings("data/hyperop_loop")

### Optimisation Bayésienne (explication brève)


L'optimisation bayésienne est une technique pour trouver le minimum $x$ une fonction inconnue $f(x)$ qui est prend beaucoup de temps à évaluer.

Dans notre cas $x$ est l'ensemble des hyper-paramètres, et $f(x)$ est le score d'un entrainement complet (la best-loss).


L'idée est d'avoir un modèle de fonction $\hat f$. Ajuster ce modèle à partir de quelques points $x_i,y_i=f(x_i)$. Puis tirer de nouveaux points $x_i,y_i=f(x_i)$ dans la zone où la fonction $\hat f$ est assez petite. Ajuster mieux la fonction $\hat f$ avec ces nouveaux points puis recommencer.



##Le défi prog


* Modifiez agent-mini pour faire des entrainements au 'chrono': chaque hyper-paramètre doit bénéficier approximativement du même temps d'entrainement.


* Une amélioration possible dans l'agent est de stocker aussi l'optimisateur-state. Ainsi si l'on redémarre l'entrainement, l'optimiseur sera déjà échauffé.




* Cette agent 'Mini' est fait pour être copié-coller dans vos différents projets et améliré. Par exemple, il pourrait aussi stocké le temps pris par l'entrainement. Et on pourrait alors ajouter un score 'loss*temps-d'entrainement' qui serait plus honnète pour les petits modèles. Ou alors on peut faire les entrainements au chrono (ex: 1 minute chacun).


Choisissez l'item ci-dessus qui vous inspire le plus, et codez-le.



